In [1]:
# Import general libraries
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
import numpy as np

# Import data

The data is stored in a separate mat-file in the folder ["Files"](http://localhost:8888/notebooks/LCS_OECS/2D/Gridded/codes_data/data/Files). It is imported using the [scipy-library](https://scipy.org/). *x, y, U, V* can be very general as they can even contain "nan" values (=regions where the dynamical system is not defined). For oceanographic applications it is pretty common to encounter regions where U, V are not defined as these areas correspond to land.

In [2]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('./data/Files/Bickley_jet.mat')

# Assign variables
U = mat_file['u']
V = mat_file['v']
x = mat_file['x']
y = mat_file['y']
time = mat_file['t']

# Create meshgrid
X, Y = np.meshgrid(x, y)

Wall time: 44.9 ms


# Initialization

The two dimensional dynamical system is given by
\begin{equation}
\mathbf{\dot{x}}(t) = \begin{pmatrix} u(x, y, t) \\ v(x, y, t) \end{pmatrix} \label{eq: dxdt},
\end{equation} with $ \mathbf{x} \in U $ and $ t \in [t_0, t_N] $.

We first need to appropriately define the data and the parameters involved when computing the trajectories generated by the dynamical system from eq. \ref{eq: dxdt}.

In [3]:
%%time
import sys
sys.path.append("./classes")
from ipynb.fs.defs.Dynamical_System import *
DS = Dynamical_System()

Wall time: 74.8 ms


## Initialize data

We start by initializing the discrete velocity data corresponding to the example of the Bickley-jet and note that we assume that the units of the velocities U, V match with the corresponding spatial and time coordinates (x, y, time). Furthermore, the spatial coordinates must be sorted in ascending order. 

*Ncores* specifies the number of cores to be used for parallel computing. If parallel computing is undesired then set *Ncores=1* (***Note: This is not recommended as it considerably slows down the computations***).

*periodic_boundary* specifies whether the flow has periodic boundary conditions at the borders of the meshgrid *X, Y*.

The list of parameters is summarized in *params_data*. *DS_init_data(params_data)* assigns the parameters and data defined in *params_data* to the object *DS*.

In [4]:
# Number of cores to be used for parallel computing
Ncores = 4

# Incompressible/Compressible flow. {True, False}
incompressible = True

# Periodic boundary conditions
periodic_boundary_x = False
periodic_boundary_y = False
periodic_boundary = [periodic_boundary_x, periodic_boundary_y]

# List of parameters of the flow.
params_data = X, Y, time, U, V, Ncores, incompressible, periodic_boundary

# Initialize data
DS._init_data(params_data)

## Initialize dynamical system from eq. \ref{eq: dxdt}

Lagrangian coherent structures (LCS) are inherently tied to the spatial domain $ U $ and the considered finite time-interval $ [t_0, t_N] $ of the dynamical system from eq. \ref{eq: dxdt}. As a consequence the extraction of LCS from the dynamical system given in eq. \ref{eq: dxdt} additionally involves specificying the spatial domain and the finite time-interval. We encourage you to test it on your own by changing for instance the length of the finite-time interval $ [t_0, t_N] $.

*t0* and *tN* specifiy the initial and final time, whereas *dt* indicates the step size to be used for numerical integration of the lagrangian particle trajectories.

The boundaries of the spatial domain are given by *xmin, xmax/ymin, ymax* and the spacing of the meshgrid is *dx/dy*. It is recommend to use the same spacing in both the x and y-direction.

Finally, the hyperparameter *ratio_auxiliary_grid* specifies the grid-spacing used for numerically computing spatial gradients.

In [5]:
%%time

# Initial time (in days)
t0 = DS.Time[0, 0]

# Final time (in days)
tN = DS.Time[0, 15]

# time step-size
dt = 0.1

# Time array
time = np.arange(t0, tN+dt, dt)

#  x, y boundaries
xmin = np.min(DS.X)
xmax = np.max(DS.X)/20
ymin = np.min(DS.Y)/2
ymax = np.max(DS.Y)/2

# spacing of meshgrid (in degrees)
dx = np.minimum(X[0, 1]-X[0, 0], Y[1, 0]-Y[0, 0])/10
dy = dx

# x, y grid with spacing dx, dy
x_grid = np.arange(xmin, xmax+dx, dx)
y_grid = np.arange(ymin, ymax+dy, dy)

# Ratio of auxiliary grid
ratio_auxiliary_grid = 0.1

params_ds = time, x_grid, y_grid, ratio_auxiliary_grid
DS._init_ds(params_ds)

Wall time: 969 µs


# Interpolation of velocity field

As we are originally dealing with a discrete representation of the velocity-field, we need to reconstruct the velocity-field in order to be able to evaluate the right hand side of the differential equation \ref{eq: dxdt} at any given point (x, t). Details regarding the interpolation technique can be found [here](http://localhost:8888/notebooks/LCS_OECS/2D/Gridded/codes_data/classes/Dynamical_System.ipynb).

In [6]:
DS._Interpolation_velocity(method = "cubic") # or method = "linear".

# Lagrangian Coherent Structures (LCS)

In the following section we higlight the most relevant diagnostic and analytic methods to extract hyperbolic, parabolic and elliptic LCS from originally discrete velocity data.

## Hyperbolics LCS

### [Finite Time Lyapunov Exponent (FTLE)](http://localhost:8888/notebooks/LCS_OECS/2D/Gridded/codes_data/classes/LCS.ipynb)

In [ ]:
FTLE_min, FTLE_max = DS._FTLE_()

=================Trajectories over meshgrid=================


  0%|          | 0/96 [00:00<?, ?it/s]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.8min


##### Advected FTLE-field

In [ ]:
from IPython.display import HTML
    
c = DS.FTLE[:,:,1].ravel()
x = DS.trajectory_grid[:,:,0,0].ravel()
y = DS.trajectory_grid[:,:,1,0].ravel()

steps = np.arange(0, DS.lenT, 1)

plt.ioff()
# create figure and axes 
fig, ax = plt.subplots(1, figsize=(8, 4))
scat = ax.scatter(x, y, c=c, s=6)
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
fig.colorbar(scat)
    
def update_plot(i, X, Y, scat):
    x_data = X[:,:,i].ravel()
    y_data = Y[:,:,i].ravel()
    ax.set_title(f'$FTLE_{{{int(DS.t0)}d}}^{{{int(DS.tN)}d}}(t = {{{np.around(DS.time[i], 2)}d}})$')
    ax.set_xlim(np.nanmin(x_data), np.nanmax(x_data))
    ax.set_ylim(np.nanmin(y_data), np.nanmax(y_data))
    scat.set_offsets(np.column_stack([x_data, y_data]))
    return scat,
    
# the FuncAnimation function iterates through our animate function using the steps array
ani = animation.FuncAnimation(fig, update_plot, frames=steps,
                                  fargs=(DS.trajectory_grid[:,:,0,:], DS.trajectory_grid[:,:,1,:], scat), 
                                  interval = 50, 
                                  blit = True)
plt.close(fig)
plt.ion()

HTML(ani.to_jshtml())

## Elliptic LCS

### [Polar Rotation Angle](http://localhost:8888/notebooks/LCS_OECS/2D/Gridded/codes_data/classes/LCS.ipynb)

In [ ]:
PRA = DS._PRA_()

#####  Advected PRA

In [ ]:
c = PRA.ravel()
x = DS.trajectory_grid[:,:,0,0].ravel()
y = DS.trajectory_grid[:,:,1,0].ravel()

steps = np.arange(0, DS.lenT, 1)

plt.ioff()
# create figure and axes 
fig, ax = plt.subplots(1, figsize=(8, 4))
scat = ax.scatter(x, y, c=c, s=6)
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
fig.colorbar(scat)
    
def update_plot(i, X, Y, scat):
    x_data = X[:,:,i].ravel()
    y_data = Y[:,:,i].ravel()
    ax.set_title(f'$PRA_{{{int(DS.t0)}d}}^{{{int(DS.tN)}d}}(t = {{{np.around(DS.time[i], 2)}d}})$')
    ax.set_xlim(np.nanmin(x_data), np.nanmax(x_data))
    ax.set_ylim(np.nanmin(y_data), np.nanmax(y_data))
    scat.set_offsets(np.column_stack([x_data, y_data]))
    return scat,
    
# the FuncAnimation function iterates through our animate function using the steps array
ani = animation.FuncAnimation(fig, update_plot, frames=steps,
                                  fargs=(DS.trajectory_grid[:,:,0,:], DS.trajectory_grid[:,:,1,:], scat), 
                                  interval = 50, 
                                  blit = True)
plt.close(fig)
plt.ion()

HTML(ani.to_jshtml())